In [1]:
import requests
import csv 
from collections import Counter
from pandas import *

listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    s=requests.session()  #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',} #send the login data
    s.post("https://www.icheckmovies.com/login/", data=login_data) #csv link 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(toplistnum) is True:
            toplistnum=int(row[5])
        toplist.append(toplistnum)        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

def process_club_list(user_list,others):
    merge_frames = others.merge(user_list,on=['title','most_checked'],how='left')
    not_same_titles = merge_frames[merge_frames.top_lists_y.isnull()]
    del not_same_titles['top_lists_y']
    not_same_titles.columns=['most_checked', 'title', 'top_lists']
    return(not_same_titles)

def isinlist(fromlist,tolist):
    fromlist=fromlist['title'].tolist()
    tolist=tolist['title'].tolist()
    if set(fromlist).issubset(set(tolist)):
        return True
    else:
        return False
    
def iseven(number):
    if number %2==0:
        return True
    else:
        return False

def lookupinlist(newtitle):
    if newframe.loc[newframe['title'] == newtitle]:
        return True
    else:
        return False

def final_table(club_list,length):
    by_checked=club_list.sort(['most_checked'],ascending=False)
    by_toplists=club_list.sort(['top_lists'],ascending=False)
    finaldata=DataFrame(columns=('most_checked','title','top_lists'))
    for counter in range(0, length):
        if iseven(counter) is True:
            rec = by_checked.iloc[[counter/2]]
            if counter == 0:
                finaldata=finaldata.append(rec,ignore_index=True)
            else:
                if isinlist(rec,finaldata) is True:
                    continue
                else:
                    finaldata=finaldata.append(rec,ignore_index=True)
        else:
            rec = by_toplists.iloc[[(counter-1)/2]]
            if isinlist(rec,finaldata) is True:
                continue
            else:
                finaldata=finaldata.append(rec,ignore_index=True)
    return finaldata


def Listmaker_pro(usr1,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    pw=open('password.txt','r')
    password=pw.read()
    csvgetter(username1,password,username2,username3)
    user1_list=listmaker(username1)
    user2_list=listmaker(username2)
    user3_list=listmaker(username3)
    combo_1_2=user1_list.append(user2_list)
    seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]
    combo_1_3=user1_list.append(user3_list)
    seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]
    combo_2_3=user2_list.append(user3_list)
    seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
    club1_list=process_club_list(user1_list,seenby_2_3) 
    club2_list=process_club_list(user2_list,seenby_1_3)
    club3_list=process_club_list(user3_list,seenby_1_2)
    output1=final_table(club1_list,20)
    output2=final_table(club2_list,20)
    output3=final_table(club3_list,20)
    return output1
    return output2
    return output3

In [5]:
listus=Listmaker_pro('hermesmovies','Haessar','James88')

There are 9 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv


In [6]:
print(listus)

    most_checked                               title  top_lists
0          42306                            Twilight          1
1          10371                      The Wicker Man         12
2          40306                       Sleepy Hollow          3
3          15991                    12 Years a Slave         12
4          36836                             Twister          2
5          35589                          The Others          6
6          26329              Låt den rätte komma in         12
7          34511                            Face/Off          4
8           6588                        Withnail & I         11
9          33617  Terminator 3: Rise of the Machines          1
10         23016                      Eyes Wide Shut         10
11         33511                            Godzilla          1
12         25688       The Rocky Horror Picture Show          9
13         31189                           Zoolander          3
14         20445                        

In [39]:
nus=listus
check=nus.to_html()
newone=check.encode('utf-8')

In [45]:
emailpw=open('emailpw','r')
pwdmail=emailpw.read()

import smtplib
from email.mime.text import MIMEText 

msg=MIMEText(newone, 'html')
fromaddr = 'ICMmovielist@gmail.com'
toaddrs  = '@gmail.com'


username = 'ICMmovielist'

server = smtplib.SMTP('smtp.gmail.com:587')
server.starttls()
server.login(username,pwdmail)
server.sendmail(fromaddr, toaddrs, msg.as_string())
server.quit() 

(221, '2.0.0 closing connection b137sm6272379qhc.18 - gsmtp')